In [ ]:
%%capture
!pip3 install seaborn

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
estatutos = "terminos_contexto_generado_inst_1shot_limpio.csv"
eurovoc = "eurovoc_contexto_generado_llama8b_inst_3shot_limpio.csv"
gold="gold_standard_enlazado.csv"


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar los archivos CSV
df_plats = pd.read_csv(estatutos, encoding='utf-8')
df_jap = pd.read_csv(eurovoc, encoding='utf-8')
gold_standard = pd.read_csv(gold, encoding='utf-8')

# Limpiar nombres de columnas
df_plats.columns = df_plats.columns.str.strip()
df_jap.columns = df_jap.columns.str.strip()
gold_standard.columns = gold_standard.columns.str.strip()


# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


# Preparar los textos combinando término y contexto
messages_plats = df_plats["Termino"].astype(str).str.cat(df_plats["Ventana_de_contexto"].astype(str), sep=",")
messages_jap = df_jap["Término"].astype(str).str.cat(df_jap["Ventana_de_contexto"].astype(str), sep=",")

# Generar embeddings
embeddings_plats = embed(messages_plats.tolist()).numpy()
embeddings_jap = embed(messages_jap.tolist()).numpy()

# Calcular similitud coseno
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)

# Umbral de similitud
umbral = 0.8

# Crear lista de matches detectados
detected_matches = []

for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_term = df_plats['Termino'].iloc[i]
            jap_term = df_jap['Término'].iloc[j]
            detected_matches.append((plats_term, jap_term, similarity))

# Convertir gold-standard a set de tuplas
gold_pairs = set(zip(gold_standard['Termino_1'], gold_standard['Término_2']))

# Convertir los matches detectados a tuplas (sin similitud)
detected_pairs = set((a, b) for a, b, _ in detected_matches)

# Evaluación
y_true = []
y_pred = []

# Evaluar solo sobre los pares detectados
for pair in detected_pairs:
    if pair in gold_pairs:
        y_true.append(1)
    else:
        y_true.append(0)
    y_pred.append(1)  # Todos los detectados son positivos

# Añadir los positivos del gold_standard que no fueron detectados (para recall)
for pair in gold_pairs:
    if pair not in detected_pairs:
        y_true.append(1)
        y_pred.append(0)

# Calcular métricas
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Mostrar los matches encontrados
print("\nMatches detectados con similitud > umbral:")
for plats_term, jap_term, similarity in detected_matches:
    print(f"Estatutos: {plats_term} | Eurovoc: {jap_term} | Similitud: {similarity:.4f}")

Detected Matches: 109
Precision: 0.1468
Recall: 0.2540
F1 Score: 0.1860

Matches detectados con similitud > umbral:
Estatutos: trabajo nocturno | Eurovoc: trabajo nocturno | Similitud: 0.8500
Estatutos: trabajo nocturno | Eurovoc: trabajo a distancia | Similitud: 0.8067
Estatutos: trabajo nocturno | Eurovoc: Derecho del trabajo | Similitud: 0.8183
Estatutos: trabajo nocturno | Eurovoc: Derecho del trabajo | Similitud: 0.8214
Estatutos: trabajo nocturno | Eurovoc: horario de trabajo | Similitud: 0.8131
Estatutos: maternidad | Eurovoc: maternidad | Similitud: 0.8869
Estatutos: maternidad | Eurovoc: personal | Similitud: 0.8164
Estatutos: maternidad | Eurovoc: reciclaje profesional | Similitud: 0.8111
Estatutos: maternidad | Eurovoc: salud pública | Similitud: 0.8294
Estatutos: maternidad | Eurovoc: religión | Similitud: 0.8550
Estatutos: maternidad | Eurovoc: Derecho del trabajo | Similitud: 0.8110
Estatutos: maternidad | Eurovoc: representación del personal | Similitud: 0.8182
Estatutos

In [ ]:
# Imprimir el porcentaje de matches del gold-standard que han superado el umbral
matches_en_gold = len(gold_pairs)
matches_detectados_en_gold = len([pair for pair in detected_pairs if pair in gold_pairs])
porcentaje_matches = (matches_detectados_en_gold / matches_en_gold) * 100 if matches_en_gold > 0 else 0
print(f"Porcentaje de matches del gold-standard superados el umbral: {porcentaje_matches:.2f}%")

Porcentaje de matches del gold-standard superados el umbral: 25.40%
